In [ ]:
#All imports
from ultralytics import YOLO
import numpy as np
import torch

In [ ]:
#Image evaluation function

def evaluate_picture(path_to_picture):
    #define the model
    model = YOLO('yolov8m-seg.pt')
    
    #Evaluate the picture on the model
    res = model(path_to_picture, imgsz=800)
    
    #create list mask
    mask_list = res[0].boxes.cls == 47

    if ~np.any(np.array(mask_list)):
        return (0,0,0,0)
    
    #extract bounding boxes and segments
    if torch.eq(mask_list, torch.Tensor([True])):
        all_boxes = np.array(res[0].boxes.xywh, dtype=object)
        all_segments = np.array(res[0].masks.xy, dtype=object)
    else:
        all_boxes = np.array(res[0].boxes.xywh, dtype=object)[mask_list]
        all_segments = np.array(res[0].masks.xy, dtype=object)[mask_list]

    #Calculate number of detected apples
    num_detected_apples = len(all_boxes)

    #Calculate largest bounding box
    max_boundingbox_width = 0;
    max_boundingbox_height = 0;

    for i in range(len(all_boxes)):
        #Width times height
        width = all_boxes[i][2]
        height = all_boxes[i][3]
        if width > max_boundingbox_width:
            max_boundingbox_width = width
        if height > max_boundingbox_height:
            max_boundingbox_height = height

    #calculate number of unique segmented pixels
    unique = set()
    for i in range(len(all_segments)):
        for j in range(len(all_segments[i])):
            element = (all_segments[i][j][0], all_segments[i][j][1])
            unique.add(element)
    num_segmented_pixels = len(unique)
    
    #return all relevant data in a tuple
    return (num_detected_apples, round(max_boundingbox_width), round(max_boundingbox_height), num_segmented_pixels)

In [ ]:
SVR = pickle.load("SVR_Model_FINAL.sav")

In [ ]:
prediction = SVR.predict(evaluate_picture("PATH_TO_PICTURE_HERE"))